# Lab 4.1 Dataset Challenge

In [1]:
# Importing packages
from IPython.display import Image 
import kagglehub
import matplotlib.pyplot as plt
import os
import polars as pl
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.regression import MeanSquaredError

2025-02-03 12:10:34.017778: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 12:10:34.025661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738613434.034299  261257 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738613434.036848  261257 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 12:10:34.047188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Start tensorboard for logging on port 6006
%load_ext tensorboard
%tensorboard --logdir runs --bind-all

### Load Dataset

In [3]:
# Download latest version
path = kagglehub.dataset_download("fedesoriano/stellar-classification-dataset-sdss17")
df = pl.read_csv(os.path.join(path, "star_classification.csv"), infer_schema_length=10000)
model_path = 'models'
train_path = 'runs'
df.head()

obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,str,f64,i64,i64,i64
1.2377e18,135.689107,32.494632,23.87882,22.2753,20.39501,19.16573,18.79371,3606,301,2,79,6.5438e18,"""GALAXY""",0.6347936,5812,56354,171
1.2377e18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.1760e19,"""GALAXY""",0.779136,10445,58158,427
1.2377e18,142.18879,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.1522e18,"""GALAXY""",0.6441945,4576,55592,299
1.2377e18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.2501,4192,301,3,214,1.0301e19,"""GALAXY""",0.9323456,9149,58039,775
1.2377e18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.8919e18,"""GALAXY""",0.1161227,6121,56187,842


### Data Preprocessing

In [4]:
# Set random state for reproducibility
random_state = 42
torch.manual_seed(random_state)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Hyperparameters
test_size = 0.10
batch_size = 64

In [6]:
# Separate data and labels, dropping irrelevant/ID features to predicting class
not_features = ["class", "obj_ID", "run_ID", "rerun_ID", "cam_col", "field_ID", "spec_obj_ID", "plate", "MJD", "fiber_ID"]
X_df, y_df_str = df.drop(not_features), df["class"]

# Convert labels to enum for numeric computation
labels = pl.Enum(["GALAXY", "STAR", "QSO"])
y_df = pl.Series(y_df_str, dtype=labels)

# Min max normalization
X_df = X_df.select((pl.all() - pl.all().min()) / (pl.all().max() - pl.all().min()))
X_df = X_df.cast(pl.Float32)  # Convert data to 32-bit precision

# Create train-test-validation split
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df.to_physical(), test_size=test_size, random_state=random_state)
X_train, X_validate, y_train, y_validate = train_test_split(
    X_train, y_train, test_size=test_size/(1 - test_size), random_state=random_state)

# Create a dataloader 
X_train_tensor, y_train_tensor = X_train.to_torch().to(device), y_train.to_torch().to(device)
X_test_tensor, y_test_tensor = X_test.to_torch().to(device), y_test.to_torch().to(device)
X_validate_tensor, y_validate_tensor = X_validate.to_torch().to(device), y_validate.to_torch().to(device)
train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=batch_size, 
    shuffle=True)

X_df.head()

alpha,delta,u,g,r,i,z,redshift
f32,f32,f32,f32,f32,f32,f32,f32
0.376905,0.503802,0.999113,0.99907,0.535344,0.427665,0.998944,0.091831
0.402286,0.491812,0.999202,0.999126,0.646203,0.515986,0.999225,0.112389
0.39496,0.534139,0.999251,0.999109,0.546218,0.435729,0.998959,0.09317
0.940947,0.1806,0.998939,0.99922,0.596946,0.486717,0.99899,0.13421
0.959118,0.392679,0.99867,0.998602,0.337999,0.287021,0.99862,0.017959


In [7]:
y_df.unique(), y_df.unique_counts()

(shape: (3,)
 Series: 'class' [enum]
 [
 	"GALAXY"
 	"STAR"
 	"QSO"
 ],
 shape: (3,)
 Series: 'class' [u32]
 [
 	59445
 	18961
 	21594
 ])

### Helper Functions

In [8]:
def calc_accuracy(model: nn.Module, X: torch.Tensor, y: torch.Tensor) -> float:
    model.eval()  # set model to evaluation mode, disable certain features
    with torch.no_grad():  # disable gradient calculations
        z = model(X)
        preds = torch.argmax(z, dim=-1)
        return (preds == y).float().mean().item()

def calc_error(model: nn.Module, X: torch.Tensor, y: torch.Tensor) -> float:
    model.eval()  # set model to evaluation mode, disable certain features
    with torch.no_grad():  # disable gradient calculations
        z = model(X)
        preds = torch.argmax(z, dim=-1)
        mse = MeanSquaredError().to(device)
        return mse(preds, y).item()

def eval_model(
    model: nn.Module, 
    model_id: int,
    X_train: torch.Tensor, 
    y_train: torch.Tensor,
    X_test: torch.Tensor, 
    y_test: torch.Tensor,
) -> None:
    print("+-------------+---------------------+")
    print(f"| model       | {model_id} |")
    print("+-------------+---------------------+")
    print(f"| train acc   | {calc_accuracy(model, X_train, y_train):.17f} |")
    print(f"| test acc    | {calc_accuracy(model, X_test, y_test):.17f} |")
    print(f"| train error | {calc_error(model, X_train, y_train):.17f} |")
    print(f"| test error  | {calc_error(model, X_test, y_test):.17f} |")
    print("+-------------+---------------------+")
    
def train_model(
    model: nn.Module, 
    opt: torch.optim.Optimizer,
    dataloader: DataLoader,
    loss_fn: nn.Module = nn.CrossEntropyLoss(),
    epochs: int = 100,
    plot_every: int = 1,
) -> int:
    unique_model_no = abs(hash((str(model), str(opt), str(loss_fn))))
    with SummaryWriter(os.path.join(train_path, f"model_{unique_model_no}")) as writer:
        model.train()  # set model to training mode

        n_total_steps = len(dataloader)

        # create metrics to display while training
        running_loss = 0.0
        running_correct = 0
        
        for epoch in range(epochs):
            for (X_batch, y_batch) in dataloader:
                opt.zero_grad()
    
                z = model(X_batch)
                loss = loss_fn(z,y_batch)
    
                loss.backward()
                opt.step()
    
                # calculate metrics
                running_loss = loss.item()
                predicted = torch.argmax(z, dim=-1)
                running_correct += (predicted == y_batch).sum().item()

                # logging and saving model
                if (epoch+1) % plot_every == 0:
                    step = epoch * n_total_steps + epoch 
                    
                    path = os.path.join(model_path, f"model_{unique_model_no}", f"{step}")
                    os.makedirs(os.path.dirname(path), exist_ok=True)
                    torch.save(model.state_dict(), path)
                    
                    writer.add_scalar('training loss', running_loss / plot_every, step)
                    writer.add_scalar('accuracy', running_correct / plot_every, step)
                    
                    running_loss = 0.0
                    running_correct = 0
                    
    return unique_model_no

### Training 

Training a simple linear model on data, using standard hyperparameter tuning

In [9]:
# Create and train model
simple_nn = nn.Sequential(
    nn.Linear(8, 6),
    nn.ReLU(),
    nn.Linear(6, 6),
    nn.ReLU(),
    nn.Linear(6, 6),
    nn.ReLU(),
    nn.Linear(6, 3)
).to(device)

lr = 1e-2
opt = torch.optim.SGD(simple_nn.parameters(), lr=lr)

model_id_0 = train_model(simple_nn, opt, train_loader)

os.system('spd-say "training over"')  # Text-to-speech to tell me when training is over so I can clean my room lol
eval_model(simple_nn, model_id_0, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

+-------------+---------------------+
| model       | 2711907495640221595 |
+-------------+---------------------+
| train acc   | 0.95538747310638428 |
| test acc    | 0.95419996976852417 |
| train error | 0.13881249725818634 |
| test error  | 0.13879999518394470 |
+-------------+---------------------+


Pretty good, but let's try and do better with regularization and other techniques. Not sure if we'll get there but we'll see.

Notes for interpreting model training performance:
- Accuracy: Higher the better
- Loss: Want smooth inverse curve converging at low loss
- Underfitting: high bias, low variance, high train error, high test error
- Overfitting: low bias, high variance, low train error, high test error

In [11]:
# Adding momentum, decrease learning rate
nn_1 = nn.Sequential(
    nn.Linear(8, 6),
    nn.ReLU(),
    nn.Linear(6, 6),
    nn.ReLU(),
    nn.Linear(6, 6),
    nn.ReLU(),
    nn.Linear(6, 3)
).to(device)

lr = 1e-3
mu = 0.9
opt = torch.optim.SGD(nn_1.parameters(), lr=lr, momentum=mu)
model_id_1 = train_model(nn_1, opt, train_loader)

os.system('spd-say "training over"')  # Text-to-speech to tell me when training is over so I can clean my room lol
eval_model(nn_1, model_id_1, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

+-------------+---------------------+
| model       | 8331765224541759696 |
+-------------+---------------------+
| train acc   | 0.94991248846054077 |
| test acc    | 0.94559997320175171 |
| train error | 0.14155000448226929 |
| test error  | 0.14650000631809235 |
+-------------+---------------------+


This model converges much faster, and in training looks like loss is higher. But on evaluation, loss accuracy is lower than the original :(

In [12]:
# Increasing epochs, adding weight decay, switch activation function to swish
nn_2 = nn.Sequential(
    nn.Linear(8, 6),
    nn.SiLU(),
    nn.Linear(6, 6),
    nn.SiLU(),
    nn.Linear(6, 6),
    nn.SiLU(),
    nn.Linear(6, 3)
).to(device)

lr = 1e-3
epochs = 300  # aim for much higher to find best model
mu = 0.9
l2_reg = 1e-4
opt = torch.optim.SGD(nn_2.parameters(), lr=lr, momentum=mu, weight_decay=l2_reg)
model_id_2 = train_model(nn_2, opt, train_loader, epochs=epochs)

os.system('spd-say "training over"')  # Text-to-speech to tell me when training is over so I can clean my room lol
eval_model(nn_2, model_id_2, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

+-------------+---------------------+
| model       | 3467952820851895034 |
+-------------+---------------------+
| train acc   | 0.96274995803833008 |
| test acc    | 0.96319997310638428 |
| train error | 0.12582500278949738 |
| test error  | 0.12409999966621399 |
+-------------+---------------------+


This model, by training longer and adding more regularization, finally outperforms the other two! Let's see the training graphs

In [20]:
Image(url=os.path.join("graphs", "lab_4.1_key.png")) 

In [18]:
Image(url=os.path.join("graphs", "lab_4.1_loss.png")) 

In [19]:
Image(url=os.path.join("graphs", "lab_4.1_accuracy.png")) 

And a final validation evaluation

In [25]:
print(calc_accuracy(simple_nn, X_validate_tensor, y_validate_tensor))
print(calc_accuracy(nn_1, X_validate_tensor, y_validate_tensor))
print(calc_accuracy(nn_2, X_validate_tensor, y_validate_tensor))

0.9535999894142151
0.9506999850273132
0.9624999761581421


## Success! regularization increased accuracy!